# Connection à la base de données 'fcd'

In [4]:
import pandas as pd
import pymongo

client = pymongo.MongoClient('localhost')
collection = client.fcd.assets

print('done')

done


# Lecture dans la base

In [5]:
cursor = collection.find({},{'asset_id':1})
entries = list(cursor)

entries[:]
df = pd.DataFrame(entries)
df.head()
print('\ndone')


done


# Entrée 1 : géo

In [6]:
#lon = float(input("longitude -> "))
#lat = float(input("latitude -> "))
rad = float(input("rayon (km) -> "))

print('\ndone')

rayon (km) -> 10

done


# Entrée 2 : date

In [7]:
from datetime import timedelta
from datetime import datetime

year = int(input("Annee -> "))
month = int(input("Mois -> "))
day = int(input("Jour -> "))
hour = int(input("Heure -> "))
minutes = int(input("Minutes -> "))
seconds = int(input("Secondes -> "))
date = datetime(year, month, day, hour, minutes, seconds)

print('\ndone')

Annee -> 2020
Mois -> 3
Jour -> 5
Heure -> 14
Minutes -> 30
Secondes -> 5

done


# Entrée 3 : période

In [8]:
T = int(input("Période (minutes) -> "))
delta = timedelta(minutes = float(T/2))

datemin = date - delta
datemax = date + delta

datemin = datetime.timestamp(datemin)
datemax = datetime.timestamp(datemax)

print('\ndone')

Période (minutes) -> 60

done


# QUERY 1 

In [9]:
import pprint


# QUERY 1
# Calculer la densité [nombre de points] sur une Zone géographique Z et une période T 

def density_list(lon, lat, rad, datemin, datemax, option,zone,periode):
    query01={}
    query02={}
    dico={}
    center=[-1,-1]
    # points dans une zone Z
    if zone:
        center=[lon,lat]
        query01 = {
            "location.geo": {
                "$nearSphere": {
                    "$geometry": {
                        "type": "Point",
                        "coordinates": [ lon, lat]
                    },
                    "$minDistance": 0,
                    "$maxDistance": rad*1000 # mètres
                }
            }
        }

    # points à une période T
    if periode :
        query02 = {'$and': [{'recorded_at': {"$gt": datemin}}, {'recorded_at': {"$lt": datemax}}]}


    # requête finale
    query = {'$and': [query01, query02]}

    if option != {} :
        query = {'$and': [query, option]}
        
    liste=list(collection.find(query))
    dico['liste_points']=liste
    dico['density']=len(liste)
    dico['center']=center
    return dico


result = density_list(-113.584, 53.5124, rad, datemin, datemax, {},True,True)
print ("density :", result['density'])

print('done')

density : 27
done


# QUERY 2

In [15]:
import pprint

# QUERY 2
# Retourner le nombre de points voisins d'un asset X dans une période T.


def asset_density(id_asset, rad, datemin, datemax):

    # les points d'un asset X
    query01 = {"asset_id": id_asset}

    # points à une période T
    query02 = {'$and': [{'recorded_at': {"$gt": datemin}}, {'recorded_at': {"$lt": datemax}}]}

    # les points d'un asset X à une période T
    query03 = {'$and': [query01, query02]}

    tmp = list(collection.find(query03))

    lon = 0
    lat = 0
    density = 0
    result = {}
    res = None

    for doc in tmp:
        lon = doc['location']['geo']['coordinates'][0]
        lat = doc['location']['geo']['coordinates'][1]
        d_list = density_list(lon, lat, rad, datemin, datemax, {"asset_id": {"$ne": id_asset}})

        if len(d_list) > density:
            density = len(d_list)
            res = doc

    if len(tmp) > 0 and density > 0:
        result['asset_id'] = id_asset
        result['density'] = density
        result['radius'] = rad
        result['center'] = res['location']['geo']['coordinates']

    return result
    
    
result = asset_density('14429000019112', rad, datemin, datemax)

if result != {}:
    print('density :', result['density'])
    print('circle zone of',result['radius'],'km around', result['center'])

print('\ndone')

density : 267
circle zone of 10.0 km around [-113.493, 53.4275]

done


# QUERY 3

In [17]:
# QUERY 3
# Retournez le top k des périodes/ zones les plus denses


def top_k_zones(k, rad, datemin, datemax):
    
    # list des id des différents asset
    ids_asset = list(collection.distinct("asset_id"))

    zones = []

    for id_ in ids_asset:

        zone = asset_density(id_, rad, datemin, datemax)

        if zone != {}:
            zones.append(zone)


    zones = sorted(zones, key=lambda zone:zone['density'], reverse=True)
    
    if k > len(zones):
        k = len(zones)
        
    return zones[0:k]

zones = top_k_zones(3, rad, datemin, datemax)
pprint.pprint(zones)
print('\ndone')

[{'asset_id': '14429000008909',
  'center': [-114.034, 50.9482],
  'density': 434,
  'radius': 10.0},
 {'asset_id': '14429000032073',
  'center': [-113.971, 50.9409],
  'density': 432,
  'radius': 10.0},
 {'asset_id': '14429000034814',
  'center': [-114.036, 50.8789],
  'density': 379,
  'radius': 10.0}]

done


# Query4

In [ ]:
def top_k_asset(id_asset,k,rad,datemin,datemax,zone,periode):
    query01 = {"asset_id": id_asset}
    tmp = list(collection.find(query01))
    lon = 0
    lat = 0
    d_list=[]
    for doc in tmp:
        lon = doc['location']['geo']['coordinates'][0]
        lat = doc['location']['geo']['coordinates'][1]
        d_list.append(density_list(lon, lat, rad, datemin, datemax,{},zone,periode))
    zones = sorted(d_list, key=lambda zone:zone['density'], reverse=True)
    if k > len(zones):
        k = len(zones)
        
    return zones[0:k]
def top_k_zones_Periode(k, rad, datemin, datemax,zone,periode):
    
    # list des id des différents asset
    ids_asset = list(collection.distinct("asset_id"))
    d_list=[]
    tmp=None
    zones = []
    tmp=top_k_asset(ids_asset[0],k,rad,datemin,datemax,zone,periode)
    for elt in tmp :
        d_list.append(elt)
    zones = sorted(d_list, key=lambda zone:zone['density'], reverse=True)
    
    if k > len(zones):
        k = len(zones)
        
    return zones[0:k]

zones = top_k_zones_Periode(3, rad, datemin, datemax,True,False)
pprint.pprint(zones)
print('\ndone')

# Query 5

In [10]:
client = pymongo.MongoClient('localhost')
collection = client.fcd.trips
zones = top_k_zones_Periode(3, rad, datemin, datemax,False,True)
pprint.pprint(zones)
print('\ndone')

NameError: name 'top_k_zones_Periode' is not defined